# Readme
Basically this data cleaning script first transform the dataset into student_id as index (row)
e.g.             quarter 1   quarter 2 ...    professor  'quarter 1 score' 'quarter 2 score' ...
student 1
student 2
student ...

After this, do data cleaning by filling nan and empty, change format etc. for encoding. 

And then added a cumulative score for each student's schedule.

In [1]:
import pandas as pd

file_path = 'final_schedules_with_score.csv'
data = pd.read_csv(file_path)

data['student_id'] = data.index // 12

restructured_data = data.pivot(index='student_id', columns='quarter', values=['courses', 'professors', 'score'])

restructured_data.columns = [f'{col[0]}_q{col[1]}' for col in restructured_data.columns]

restructured_data = restructured_data.reset_index()

output_path = 'schedule_with_score_transformed.csv'
restructured_data.to_csv(output_path, index=False)

print(f'Transformed data saved as {output_path}')
restructured_data.head()

Transformed data saved as schedule_with_score_transformed.csv


,student_id,courses_q1,courses_q2,courses_q3,courses_q4,courses_q5,courses_q6,courses_q7,courses_q8,courses_q9,...,score_q3,score_q4,score_q5,score_q6,score_q7,score_q8,score_q9,score_q10,score_q11,score_q12
0,0,"['MATH 19A', 'CSE 20']","['MATH 19B', 'CSE 30']","['AM 30', 'MATH 21']","['ECE 30', 'CSE 16']","['CSE 107', 'CSE 40']",['CSE 12'],['CSE 13S'],"['CSE 101', 'CSE 120']","['CSE 101M', 'CSE 102']",...,0,[],[],[],[],[],[],[],[],[]
1,1,"['MATH 20A', 'CSE 20']","['MATH 20B', 'CSE 16']","['AM 30', 'ECE 30']","['STAT 131', 'CSE 12']","['AM 10', 'CSE 13S']","['CSE 120', 'CSE 30']","['CSE 40', 'CSE 101']","['CSE 185S', 'CSE 114A']","['CSE 130', 'CSE 101M']",...,2.5,[],[],[],[],[],[],[],[],[]
2,2,"['MATH 20A', 'CSE 195']","['MATH 20B', 'CSE 20']","['AM 30', 'CSE 30']","['AM 10', 'ECE 30']","['CSE 40', 'CSE 12']","['CSE 16', 'CSE 13S']","['CSE 101', 'CSE 107']","['CSE 114A', 'CSE 130']","['CSE 103', 'CSE 120']",...,2.5,[],[],[],[],[],[],[],[],[]
3,3,"['MATH 19A', 'CSE 20']","['MATH 19B', 'CSE 30']","['MATH 23A', 'ECE 30']","['CSE 40', 'CSE 12']","['AM 10', 'CSE 13S']","['CSE 120', 'CSE 16']","['CSE 101', 'CSE 107']","['CSE 114A', 'CSE 130']","['CSE 102', 'CSE 101M']",...,2.5,[],[],[],[],[],[],[],[],[]
4,4,"['MATH 20A', 'CSE 20']","['MATH 20B', 'CSE 12']","['MATH 23A', 'AM 10']","['CSE 185E', 'ECE 30']","['CSE 16', 'CSE 30']","['CSE 40', 'CSE 107']",['CSE 13S'],"['CSE 101', 'CSE 120']","['CSE 114A', 'CSE 101M']",...,-0.5999999999999996,[],[],[],[],[],[],[],[],[]


In [2]:
import pandas as pd
import ast

file_path = 'schedule_with_score_transformed.csv'
data = pd.read_csv(file_path)

# 1. Handle missing values in courses, professors, and scores
for q in range(1, 13):
    # Fill empty courses/professors with placeholders if empty
    data[f'courses_q{q}'] = data[f'courses_q{q}'].apply(lambda x: [] if pd.isna(x) or x == '[]' else ast.literal_eval(x))
    data[f'professors_q{q}'] = data[f'professors_q{q}'].apply(lambda x: [] if pd.isna(x) or x == '[]' else ast.literal_eval(x))
    
    # replace null val scores with 0
    data[f'score_q{q}'] = pd.to_numeric(data[f'score_q{q}'], errors='coerce').fillna(0)

# 2. Standardize format for courses and professors
# Convert any string lists to actual lists
for q in range(1, 13):
    data[f'courses_q{q}'] = data[f'courses_q{q}'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    data[f'professors_q{q}'] = data[f'professors_q{q}'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# 3. Calculate cumulative score
score_columns = [f'score_q{q}' for q in range(1, 13)]
data['cumulative_score'] = data[score_columns].sum(axis=1)

output_path = 'cleaned_schedule_with_cumulative_score.csv'
data.to_csv(output_path, index=False)

print(f'Cleaned data with cumulative score saved as {output_path}')
data.head()

Cleaned data with cumulative score saved as cleaned_schedule_with_cumulative_score.csv


,student_id,courses_q1,courses_q2,courses_q3,courses_q4,courses_q5,courses_q6,courses_q7,courses_q8,courses_q9,...,score_q4,score_q5,score_q6,score_q7,score_q8,score_q9,score_q10,score_q11,score_q12,cumulative_score
0,0,"[MATH 19A, CSE 20]","[MATH 19B, CSE 30]","[AM 30, MATH 21]","[ECE 30, CSE 16]","[CSE 107, CSE 40]",[CSE 12],[CSE 13S],"[CSE 101, CSE 120]","[CSE 101M, CSE 102]",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.6
1,1,"[MATH 20A, CSE 20]","[MATH 20B, CSE 16]","[AM 30, ECE 30]","[STAT 131, CSE 12]","[AM 10, CSE 13S]","[CSE 120, CSE 30]","[CSE 40, CSE 101]","[CSE 185S, CSE 114A]","[CSE 130, CSE 101M]",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.3
2,2,"[MATH 20A, CSE 195]","[MATH 20B, CSE 20]","[AM 30, CSE 30]","[AM 10, ECE 30]","[CSE 40, CSE 12]","[CSE 16, CSE 13S]","[CSE 101, CSE 107]","[CSE 114A, CSE 130]","[CSE 103, CSE 120]",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.6
3,3,"[MATH 19A, CSE 20]","[MATH 19B, CSE 30]","[MATH 23A, ECE 30]","[CSE 40, CSE 12]","[AM 10, CSE 13S]","[CSE 120, CSE 16]","[CSE 101, CSE 107]","[CSE 114A, CSE 130]","[CSE 102, CSE 101M]",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.7
4,4,"[MATH 20A, CSE 20]","[MATH 20B, CSE 12]","[MATH 23A, AM 10]","[CSE 185E, ECE 30]","[CSE 16, CSE 30]","[CSE 40, CSE 107]",[CSE 13S],"[CSE 101, CSE 120]","[CSE 114A, CSE 101M]",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1
